---
title: Research of existing geojson files for Germany on municipality level
date: now
author: Jan Cap
---

We found one data source of geojson files for Germany on opendatalab.de: https://opendatalab.de/projects/geojson-utilities/#
Lets try to load the state level boundaries first. Then we will try the municipality level boundaries.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

## State level boundaries load attempt

In [ ]:
# Try to load German municipality boundaries from a common source
# Let's start with a sample GeoJSON for German states (Bundesländer)

try:
    # Load GeoJSON data using geopandas
    gdf_state = gpd.read_file("../data/bundeslaender_simplify200.geojson")

    print(f"Loaded GeoDataFrame with {len(gdf_state)} features")
    print(f"Columns: {list(gdf_state.columns)}")
    print(f"CRS: {gdf_state.crs}")

    # Display first few rows
    print("\nFirst 3 rows:")
    display(gdf_state.head(3))

except Exception as e:
    print(f"Error loading from URL: {e}")
    print("Let's try a different approach...")

Data columns explanation:
- RS (Regional key): Depending on the level, 2-digit, 5-digit, or 12-digit. (2-digit for states, 5-digit for districts, 12-digit for municipalities)
- GEN (Geographical name): official name of the administrative unit
- BEZ (Official designation): official designation of the administrative unit like "Stadt", "Landkreis", etc.
- Destatis (Destatis data): includes area in square meters and population numbers

In [ ]:
# Visualize the GeoJSON data
if "gdf" in locals() and not gdf_state.empty:
    # Create a simple plot
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    gdf_state.plot(ax=ax, color="lightblue", edgecolor="black", linewidth=0.5)
    ax.set_title("German Federal States (Bundesländer)")
    ax.set_axis_off()
    plt.tight_layout()
    plt.show()

    # Show some basic statistics
    print("\nGeoDataFrame Info:")
    print(f"Shape: {gdf_state.shape}")
    print(f"Geometry type: {gdf_state.geometry.geom_type.unique()}")
    print(f"Bounds: {gdf_state.total_bounds}")
else:
    print("No geodata loaded to visualize")

## Municipality boundaries load attempt

In [ ]:
# Try to load municipality-level data (Gemeinden)
# Try to load German municipality boundaries from a common source
# Let's start with a sample GeoJSON for German states (Bundesländer)

try:
    # Load GeoJSON data using geopandas
    gdf_mun = gpd.read_file("../data/gemeinden_simplify200.geojson")

    print(f"Loaded GeoDataFrame with {len(gdf_mun)} features")
    print(f"Columns: {list(gdf_mun.columns)}")
    print(f"CRS: {gdf_mun.crs}")

    # Display first few rows
    print("\nFirst 3 rows:")
    display(gdf_mun.head(3))

except Exception as e:
    print(f"Error loading from URL: {e}")
    print("Let's try a different approach...")

Data columns explanation:
- RS (Regional key): Depending on the level, 2-digit, 5-digit, or 12-digit. (2-digit for states, 5-digit for districts, 12-digit for municipalities)
- AGS (Official municipality key): 8-digit official municipality key
- GEN (Geographical name): official name of the administrative unit
- BEZ (Official designation): official designation of the administrative unit like "Stadt", "Landkreis", etc.
- Destatis (Destatis data): includes area in square meters and population numbers

In [ ]:
gdf_mun[gdf_mun["AGS"].str.startswith("0200")]

In [ ]:
# Visualize the GeoJSON data
# Create a simple plot
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
gdf_mun.plot(ax=ax, color="lightblue", edgecolor="black", linewidth=0.5)
ax.set_title("German Municipalities (Gemeinden)")
ax.set_axis_off()
plt.tight_layout()
plt.show()

# Show some basic statistics
print("\nGeoDataFrame Info:")
print(f"Shape: {gdf_mun.shape}")
print(f"Geometry type: {gdf_mun.geometry.geom_type.unique()}")
print(f"Bounds: {gdf_mun.total_bounds}")